In [1]:
import sklearn
import numpy as np
import pickle
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

C:\Users\pinup\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\pinup\anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\pinup\anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:\n%s" %


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
from torchvision.utils import make_grid
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split
import torch.optim as optim

In [3]:
data=pd.read_excel(r"D:\NLP_telugu_data\sarcastic_sentences.xlsx")
data=data.dropna()
data['Labels']=data['Labels'].replace(to_replace ='Normal question and sarcastic answer',value='Sarcastic')
data['Labels']=data['Labels'].replace(to_replace ='Normal question and normal answer',value='Non-Sarcastic')

In [4]:
data.head()

,Sentences,Labels
0,బంగారం ఆఫీస్ నుండి ఇప్పుడే వచ్చావా? లేదు గంట అ...,Sarcastic
1,డాక్టర్ నిన్ను నాన్వెజ్ మానేయమన్నారుగా ఇంకా మా...,Sarcastic
2,అల్లుడు? ఏంటి మావయ్య. నిన్ను అడగాలంటే కోద్ధిగా...,Sarcastic
3,"నీతో మాట్లాడాలి, కొంచెం టైమ్ ఇస్తావా? అయ్యో నా...",Sarcastic
4,"కెప్టెన్, టైటానిక్ షిప్ ఇంకా ఏం స్టార్ట్ చేయలే...",Sarcastic


In [5]:
len(data['Sentences'])

1791

# Removing Punctuations

In [6]:
punctuations = '''!()-[]{};:'"+\,<>.?/@#$%^&*_~0123456789'''
rpunct=[]
for i in data.Sentences:
    no_punct = " "
    for i in i:
        if i not in punctuations:
            no_punct=no_punct+i
    rpunct.append(no_punct.strip())

# Tokenization

In [7]:
tokens=[]
for i in rpunct:
    g=i.split()
    tokens.append(g)

# StopWords Removal

In [8]:
stop_words=open(r"D:\NLP_telugu_data\telugu_stop_words.txt",'r',encoding='utf-8')

In [9]:
stop_words_list=[]
for i in stop_words.readlines():
    stop_words_list.append(i.strip())

In [10]:
for i in tokens:
    for j in i:
        if j in stop_words_list:
            i.remove(j)

# POS TAGGED PICKLE FILES

In [11]:
import pickle
import pandas as pd
import numpy as np

In [12]:
postags = pd.read_pickle(r'D:\NLP_telugu_data\postags.pkl')
tokens = pd.read_pickle(r'D:\NLP_telugu_data\tokens.pkl')
y = pd.read_pickle(r'D:\NLP_telugu_data\y.pkl')

In [13]:
sremoved_sentences=[]

for i in tokens:
    st=' '.join(i)
    sremoved_sentences.append(st)

In [14]:
sremoved_sentences[0]

'బంగారం ఆఫీస్ నుండి ఇప్పుడే వచ్చావా లేదు గంట అయ్యింది బస్ స్టాండులో జామకాయలు అమ్మి వచ్చేటప్పటికి టైమ్ అయ్యింది'

# WORD2VEC

In [ ]:
from gensim.test.utils import get_tmpfile
from gensim.models import Word2Vec
path = get_tmpfile("word2vec.model")
model = Word2Vec(tokens, window=5, min_count=1, workers=4)

In [ ]:
model.wv.vectors.shape

In [ ]:
weights=model.wv.vectors

In [ ]:
weights=torch.tensor(weights)

In [ ]:
model.wv.get_vector('బంగారం')

In [ ]:
embedding=nn.Embedding.from_pretrained(weights)

Pytorch Embedding

Loading the Dataset

Preprocessing

Tokenizing and Sampling

Building Vocabulary

Converting Data into Tensors

Exercise: Obtaining Tensor Representation

In [15]:
wordtokens=[]
for i in tokens:
    for j in i:
        wordtokens.append(j)

In [16]:
len(wordtokens)

30216

In [17]:
from collections import Counter

In [18]:
vocab=Counter(wordtokens)

In [19]:
vocab = sorted(vocab, key=vocab.get, reverse=True)
vocab_size = len(vocab)+1

In [20]:
word2idx = {word: ind+1 for ind, word in enumerate(vocab)}

In [21]:
encoded_sentences = []
for i in tokens:
    tmp=[]
    for j in i:
        tmp.append(word2idx[j])
    encoded_sentences.append(tmp)

In [22]:
c=[]
for i in encoded_sentences:
    c.append(len(i))

In [23]:
len(encoded_sentences)

1791

In [24]:
def seq_padding(encoded_sentences,max_len):
    padded_sentences=[]
    for sent in encoded_sentences:
        if len(sent)>max_len:
            padded_sentences.append(sent[:max_len])
        elif len(sent)<max_len:
            while len(sent)!=max_len:
                sent.append(0)
            padded_sentences.append(sent)
        else:
            padded_sentences.append(sent)
    return padded_sentences

In [25]:
padded_sentences=seq_padding(encoded_sentences,15)

In [26]:
padded_sentences=torch.LongTensor(padded_sentences)

In [27]:
padded_sentences[1]

tensor([  25,   39, 1489, 2331,   55, 2332,    1,  650, 1490, 1491,  463, 4236,
        4237, 2333,   76])

In [28]:
emd_dim=10

In [29]:
emb_layer = nn.Embedding(vocab_size, emd_dim,padding_idx=0)
word_vectors = emb_layer(padded_sentences[:10])

In [30]:
input_sent=padded_sentences[:10].t()

In [31]:
input_sent

tensor([[ 163,   25,  114,  313, 4244,  397,   12,   12,  114,  560],
        [ 185,   39,   26, 1060, 4245, 4249, 2339,    4,   26,   31],
        [  40, 1489, 2334,  140, 4246,  346, 2340,  151, 2344, 1064],
        [ 213, 2331,   39, 1059,   55, 4250,   33,  466, 2345, 2346],
        [ 837,   55, 4238,  396,    4,   18,  838, 2341,    5,   14],
        [   5, 2332, 4239,   94,  287, 4251,    1,  200,  314,  265],
        [ 242,    1, 4240, 4241,  465,  186,  243, 1062,   67, 4259],
        [ 229,  650,   33, 4242,    1, 4252,   35, 1063,   11,    1],
        [1488, 1490,   13, 4243,   95, 2337, 1061, 4257, 1492,   36],
        [4232, 1491,  199,    0, 4247, 4253,  839,    1,    8,  125],
        [4233,  463, 2335,    0,   59,  288, 4256,    3,  467, 2347],
        [4234, 4236, 2336,    0, 4248,   79,   40,  264, 4258,   70],
        [4235, 4237,   83,    0,    0, 2338,    0,   21,    0,  651],
        [1059, 2333,   89,    0,    0,    9,    0, 2342,    0,  152],
        [ 229,   76,

In [32]:
emb_layer = nn.Embedding(vocab_size, emd_dim)
word_vectors = emb_layer(input_sent)
rnn=nn.RNN(emd_dim,64)
fc1=nn.Linear(64,32)
fc2=nn.Linear(32,16)
fc3=nn.Linear(16,2)

In [33]:
word_vectors.shape

torch.Size([15, 10, 10])

In [34]:
hidden=torch.zeros((1, 10, 64))

In [35]:
out,hidden_out=rnn(word_vectors,hidden)

In [36]:
word_vectors.shape

torch.Size([15, 10, 10])

In [37]:
out.shape

torch.Size([15, 10, 64])

In [38]:
hidden_out.shape

torch.Size([1, 10, 64])

In [39]:
out.shape

torch.Size([15, 10, 64])

In [40]:
out=fc1(out[-1,:,:])
out=fc2(out)
out=fc3(out)

In [41]:
out

tensor([[ 0.0985, -0.2807],
        [ 0.0860, -0.2553],
        [ 0.0515, -0.3734],
        [ 0.0600, -0.2668],
        [ 0.0638, -0.2480],
        [ 0.0702, -0.2775],
        [ 0.0635, -0.2756],
        [ 0.1179, -0.2593],
        [ 0.0611, -0.2733],
        [ 0.1165, -0.1908]], grad_fn=<AddmmBackward>)

In [42]:
out=F.sigmoid(out)

C:\Users\pinup\anaconda3\lib\site-packages\torch\nn\functional.py:1805: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [43]:
out

tensor([[0.5246, 0.4303],
        [0.5215, 0.4365],
        [0.5129, 0.4077],
        [0.5150, 0.4337],
        [0.5159, 0.4383],
        [0.5175, 0.4311],
        [0.5159, 0.4315],
        [0.5294, 0.4355],
        [0.5153, 0.4321],
        [0.5291, 0.4525]], grad_fn=<SigmoidBackward>)

In [44]:
temp_y=[1,0]

In [46]:
temp_y=torch.tensor(temp_y)

In [47]:
loss=F.cross_entropy

In [48]:
torch.tensor(y[0:10])

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [49]:
lossy=loss(out,torch.tensor(y[0:10]))

In [50]:
lossy

tensor(0.7375, grad_fn=<NllLossBackward>)

In [ ]:
y=torch.tensor(y)

# Model

In [51]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(padded_sentences,y)

In [52]:
Xtrain_loader=DataLoader(X_train,batch_size=10)
ytrain_loader=DataLoader(y_train,batch_size=10)

In [53]:
Xtest_loader=DataLoader(X_test,batch_size=10)
ytest_loader=DataLoader(y_test,batch_size=10)

In [54]:
batch_size=20
hidden_dim=64
lr=0.0001
emd_dim=200

In [55]:
class Sarcasm_Detector(nn.Module):
    def __init__(self):
        super().__init__()
        self.emb_layer = nn.Embedding(vocab_size, emd_dim)
        #self.word_vectors = self.emb_layer(input_sent)
        self.hidden_layout=torch.zeros(1,batch_size,hidden_dim)
        self.rnn=nn.RNN(emd_dim,hidden_dim)
        self.fc1=nn.Linear(64,32)
        self.fc2=nn.Linear(32,16)
        self.fc3=nn.Linear(16,2)
        self.sigd=nn.ReLU()
    def forward(self,xb):
        X=xb.t()
        embeds=self.emb_layer(X)
        out,hidden_out=self.rnn(embeds)
        
        output=self.fc1(out[-1,:,:])
        output=self.fc2(output)
        output=self.fc3(output)
        output=self.sigd(output)
        return output

In [56]:
model=Sarcasm_Detector()

In [57]:
model

Sarcasm_Detector(
  (emb_layer): Embedding(9547, 200)
  (rnn): RNN(200, 64)
  (fc1): Linear(in_features=64, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=2, bias=True)
  (sigd): ReLU()
)

In [58]:
def evaluate(model,Xval_loader,yval_loader):
    loss=0
    n=0
    acc=0
    for images,outputs in zip(Xval_loader,yval_loader):
        val_preds=model(images)
        val_los=F.cross_entropy(val_preds,outputs)
        loss+=val_los
        n+=1
        _, preds = torch.max(val_preds, dim=1)
        acc+=torch.tensor(torch.sum(preds == outputs).item() / len(preds))
        
    avg_loss=loss/n
    avg_acc=acc/n
    print("epoch:",epoch+1,"val_loss: ",avg_loss.item(),"val_acc: ",avg_acc.item())

In [59]:
optimizer=optim.Adam(model.parameters(),lr)

In [60]:
epochs=10

In [61]:
for epoch in range(epochs):
    for xb,yb in zip(Xtrain_loader,ytrain_loader):
        preds=model.forward(xb)
        loss=F.cross_entropy(preds,yb)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    evaluate(model,Xtest_loader,ytest_loader)

epoch: 1 val_loss:  0.6368662714958191 val_acc:  0.7283331751823425
epoch: 2 val_loss:  0.571229100227356 val_acc:  0.8100000023841858
epoch: 3 val_loss:  0.467288076877594 val_acc:  0.8144445419311523
epoch: 4 val_loss:  0.4225701093673706 val_acc:  0.823333203792572
epoch: 5 val_loss:  0.41005653142929077 val_acc:  0.8211109638214111
epoch: 6 val_loss:  0.40933364629745483 val_acc:  0.8299998641014099
epoch: 7 val_loss:  0.40694376826286316 val_acc:  0.823333203792572
epoch: 8 val_loss:  0.39961904287338257 val_acc:  0.832222044467926
epoch: 9 val_loss:  0.40926921367645264 val_acc:  0.8277776837348938
epoch: 10 val_loss:  0.41528046131134033 val_acc:  0.832222044467926


In [ ]:
torch.save(model.state_dict(),r"D:\NLP_telugu_data\Assignment_NLP\sarcasm_model.pth")

In [ ]:
torch.save(model,r"D:\NLP_telugu_data\Assignment_NLP\sarcasm_model_entire.pth")

# GRU

In [ ]:
hidden.shape

In [ ]:
emb_layer = nn.Embedding(vocab_size, emd_dim)
word_vectors = emb_layer(input_sent)
rnn=nn.LSTM(emd_dim,64)

In [ ]:
word_vectors.shape

In [ ]:
out,hidden_out=rnn(word_vectors)

In [ ]:
out.shape

In [ ]:
hidden_out[1].shape

In [82]:
batch_size=20
hidden_dim=64
lr=0.0001
emd_dim=200

In [93]:
class Sarcasm_Detector_GRU(nn.Module):
    def __init__(self):
        super().__init__()
        self.emb_layer = nn.Embedding(vocab_size, emd_dim)
        self.hidden_layout=torch.zeros(1,batch_size,hidden_dim)
        self.rnn=nn.GRU(emd_dim,hidden_dim)
        self.fc1=nn.Linear(hidden_dim,32)
        self.fc2=nn.Linear(32,16)
        self.fc3=nn.Linear(16,2)
        self.sigd=nn.ReLU()
    def forward(self,xb):
        X=xb.t()
        embeds=self.emb_layer(X)
        out,hidden_out=self.rnn(embeds)
        output=out[-1,:,:]
        output=self.fc1(output)
        output=self.fc2(output)
        output=self.fc3(output)
        output=self.sigd(output)
        return output

In [94]:
model_GRU=Sarcasm_Detector_GRU()

In [95]:
model_GRU

Sarcasm_Detector_GRU(
  (emb_layer): Embedding(9547, 200)
  (rnn): GRU(200, 64)
  (fc1): Linear(in_features=64, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=2, bias=True)
  (sigd): ReLU()
)

In [96]:
optimizer_GRU=optim.Adam(model_GRU.parameters(),lr)

In [97]:
epochs=10

In [98]:
for epoch in range(epochs):
    for xb,yb in zip(Xtrain_loader,ytrain_loader):
        preds=model_GRU.forward(xb)
        loss=F.cross_entropy(preds,yb)
        loss.backward()
        optimizer_GRU.step()
        optimizer_GRU.zero_grad()
    evaluate(model_GRU,Xtest_loader,ytest_loader)

epoch: 1 val_loss:  0.727958083152771 val_acc:  0.2827777862548828
epoch: 2 val_loss:  0.6283553838729858 val_acc:  0.6966666579246521
epoch: 3 val_loss:  0.4773036539554596 val_acc:  0.8350000977516174
epoch: 4 val_loss:  0.3820696175098419 val_acc:  0.8394445776939392
epoch: 5 val_loss:  0.33872902393341064 val_acc:  0.8661110997200012
epoch: 6 val_loss:  0.3195364773273468 val_acc:  0.8905555605888367
epoch: 7 val_loss:  0.31059786677360535 val_acc:  0.8949999213218689
epoch: 8 val_loss:  0.3099302053451538 val_acc:  0.8950000405311584
epoch: 9 val_loss:  0.317277729511261 val_acc:  0.8927777409553528
epoch: 10 val_loss:  0.32530972361564636 val_acc:  0.8816666603088379


In [99]:
torch.save(model_GRU.state_dict(),r"D:\NLP_telugu_data\Assignment_NLP\sarcasm_model_GRU.pth")

In [100]:
torch.save(model_GRU,r"D:\NLP_telugu_data\Assignment_NLP\sarcasm_model_GRU_FULL.pth")

# LSTM

In [290]:
batch_size=10
hidden_dim=64
lr=0.0001
emd_dim=100

In [291]:
class Sarcasm_Detector_LSTM(nn.Module):
    def __init__(self):
        super().__init__()
        self.emb_layer = nn.Embedding(vocab_size, emd_dim)
        self.hidden_layout=torch.zeros(1,batch_size,hidden_dim)
        self.rnn=nn.LSTM(emd_dim,hidden_dim)
        self.fc1=nn.Linear(hidden_dim,32)
        #self.dropout=nn.Dropout(0.2)
        self.fc2=nn.Linear(32,16)
        self.fc3=nn.Linear(16,2)
        self.sigd=nn.ReLU()
    def forward(self,xb):
        X=xb.t()
        embeds=self.emb_layer(X)
        out,hidden_out=self.rnn(embeds)
        output=out[-1,:,:]
        output=self.fc1(output)
        output=self.fc2(output)
        output=self.fc3(output)
        output=self.sigd(output)
        return output

In [292]:
model_LSTM=Sarcasm_Detector_LSTM()

In [293]:
model_LSTM

Sarcasm_Detector_LSTM(
  (emb_layer): Embedding(9547, 100)
  (rnn): LSTM(100, 64)
  (fc1): Linear(in_features=64, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=2, bias=True)
  (sigd): ReLU()
)

In [294]:
optimizer_LSTM=optim.Adam(model_LSTM.parameters(),lr)

In [295]:
epochs=10

In [296]:
for epoch in range(epochs):
    for xb,yb in zip(Xtrain_loader,ytrain_loader):
        preds=model_LSTM.forward(xb)
        loss=F.cross_entropy(preds,yb)
        loss.backward()
        optimizer_LSTM.step()
        optimizer_LSTM.zero_grad()
    evaluate(model_LSTM,Xtest_loader,ytest_loader)

epoch: 1 val_loss:  0.5988768935203552 val_acc:  0.8122221827507019
epoch: 2 val_loss:  0.5289404392242432 val_acc:  0.8166667222976685
epoch: 3 val_loss:  0.40678444504737854 val_acc:  0.8305556178092957
epoch: 4 val_loss:  0.37384578585624695 val_acc:  0.8416666388511658
epoch: 5 val_loss:  0.3448254466056824 val_acc:  0.8505554795265198
epoch: 6 val_loss:  0.33886635303497314 val_acc:  0.8594444394111633
epoch: 7 val_loss:  0.33243539929389954 val_acc:  0.8772222995758057
epoch: 8 val_loss:  0.33255448937416077 val_acc:  0.8772221803665161
epoch: 9 val_loss:  0.33997586369514465 val_acc:  0.8794444799423218
epoch: 10 val_loss:  0.3519827425479889 val_acc:  0.8816665410995483


In [297]:
torch.save(model_LSTM.state_dict(),r"D:\NLP_telugu_data\Assignment_NLP\sarcasm_model_LSTM.pth")

In [298]:
torch.save(model_LSTM,r"D:\NLP_telugu_data\Assignment_NLP\sarcasm_model_LSTM_FULL.pth")